In [59]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
dr="/content/drive/My Drive/Classroom/Datasets/"

In [3]:
import os
os.listdir(dr)

['glove.6B',
 'donar_preprocess.csv',
 'donar_tfselected_words.csv',
 'chunk3_train_sparse_matrix.npz',
 'chunk2_train_sparse_matrix.npz',
 'chunk1_train_sparse_matrix.npz',
 'sample_test_sparse_matrix.npz',
 'Donesaving_reg1.csv',
 'reg_test.csv',
 'reg_train1.csv',
 'Donesaving_reg2.csv',
 'reg_train2.csv',
 'reg_train3.csv',
 'random_file_size.csv',
 'srandom_parse_matrix.npz',
 'top_feat_names.npy',
 'top_feat_indexs.npy',
 'file_name_list.npy',
 'file_size.csv',
 'samplebyte_images.rar',
 'samplebyte_images',
 'trainLabels.csv',
 'result.csv',
 'asmoutputfile.csv',
 'asm_result_with_size.csv',
 'one_data.csv',
 'final_byte.csv',
 'cnn_feat.csv',
 'IMDB_files_csv',
 'en_train.csv',
 'model_INT',
 'label_model.h5']

In [4]:
import numpy as np
import string
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
df=pd.read_csv(dr+"en_train.csv")

In [8]:
df.shape

(9918441, 5)

In [0]:
data=df[df['sentence_id']<=150000]#taking 1.5 million sentence data

In [10]:
data.shape

(1922423, 5)

In [11]:
data['class'].value_counts()

PLAIN         1418821
PUNCT          367436
DATE            54281
LETTERS         28991
CARDINAL        25820
VERBATIM        15076
MEASURE          3092
ORDINAL          2385
DECIMAL          2022
MONEY            1224
DIGIT            1011
ELECTRONIC        957
TELEPHONE         738
TIME              245
FRACTION          228
ADDRESS            96
Name: class, dtype: int64

In [0]:
data["new_class"]=data['class'].apply(lambda c: "NUMERIC" if str(c)=='DATE' or  str(c)=='CARDINAL' or  str(c)=='MEASURE' or  str(c)=='ORDINAL' or  str(c)=='DECIMAL' or  str(c)=='MONEY' or  str(c)=='DIGIT' or  str(c)=='TELEPHONE' or str(c)=='TIME' or str(c)=='FRACTION' or  str(c)=='ADDRESS' else( "PLAIN_PUNC" if str(c)=='PLAIN' or str(c)=='PUNCT' else( "LETTERS_VERB" if str(c)=='LETTERS' or str(c)=='VERBATIM' else c ) ))

In [0]:
letter_df=data[data['new_class']=='LETTERS_VERB']
df=data[data['new_class']=='NUMERIC']
df.columns=['sentence_id',	'token_id',	'class',	'written',	'spoken',	'new_class']

In [0]:

def tokenization(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [20]:
df.head()

,sentence_id,token_id,class,written,spoken,new_class
10,1,0,DATE,2006,two thousand six,NUMERIC
51,3,7,DATE,2007,two thousand seven,NUMERIC
80,5,0,DATE,2008,two thousand eight,NUMERIC
95,6,3,CARDINAL,91,ninety one,NUMERIC
111,8,1,DATE,4 March 2014,the fourth of march twenty fourteen,NUMERIC


In [0]:
df[['spoken','written']]=df[['spoken','written']].astype('str')


In [26]:
# prepare english tokenizer
eng_tokenizer = tokenization(df['spoken'])
eng_vocab_size = len(eng_tokenizer.word_index) + 1

eng_length = 20
print('Size of English Vocabulary : %d' % eng_vocab_size)

Size of English Vocabulary : 326


In [27]:
# prepare Hindi tokenizer
hin_tokenizer = tokenization(df['written'])
hin_vocab_size = len(hin_tokenizer.word_index) + 1

hin_length = 20
print('Size of Hindi Vocabulary : %d' % hin_vocab_size)

Size of Hindi Vocabulary : 4267


In [0]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

## Model building

In [0]:
df=df[['spoken','written']]

In [33]:
df.head(5)

,spoken,written
10,two thousand six,2006
51,two thousand seven,2007
80,two thousand eight,2008
95,ninety one,91
111,the fourth of march twenty fourteen,4 March 2014


In [0]:
#SPLITTING DATA
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.25, random_state = 42)
# prepare training data
trainX = encode_sequences(eng_tokenizer, eng_length, train.iloc[:,0])
trainY = encode_sequences(hin_tokenizer, hin_length, train.iloc[:,1])
# prepare validation data
testX = encode_sequences(eng_tokenizer, eng_length, test.iloc[:,0])
testY = encode_sequences(hin_tokenizer, hin_length, test.iloc[:,0])

In [35]:
trainX.shape,trainY.shape,eng_length,hin_length,eng_vocab_size,hin_vocab_size

((68356, 20), (68356, 20), 20, 20, 326, 4267)

In [36]:
#model
#ref:https://keras.io/layers/core/
#ref:https://datascience.stackexchange.com/questions/46491/what-is-the-job-of-repeatvector-and-timedistributed
model = Sequential()
model.add(Embedding(eng_vocab_size, 512, input_length=eng_length, mask_zero=True))
model.add(LSTM(512))
model.add(RepeatVector(hin_length))
model.add(LSTM(512, return_sequences=True))
model.add(Dense(hin_vocab_size, activation='softmax'))




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
#https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/
rms = optimizers.RMSprop(lr=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [50]:
#https://medium.com/@ageitgey/machine-learning-is-fun-part-5-language-translation-with-deep-learning-and-the-magic-of-sequences-2ace0acca0aa#saving model
filename = dr+'num_model.h5' 
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1 ), 
          epochs=10, batch_size=32, 
          validation_split = 0.2,
          callbacks=[checkpoint], verbose=1)

Train on 54684 samples, validate on 13672 samples
Epoch 1/10
54684/54684 [==============================] - 2273s 42ms/step - loss: 0.1193 - acc: 0.9810 - val_loss: 0.0991 - val_acc: 0.9857

Epoch 00001: val_loss improved from inf to 0.09906, saving model to /content/drive/My Drive/Classroom/Datasets/num_model.h5
Epoch 2/10
54684/54684 [==============================] - 2292s 42ms/step - loss: 0.0889 - acc: 0.9866 - val_loss: 0.0897 - val_acc: 0.9868

Epoch 00002: val_loss improved from 0.09906 to 0.08970, saving model to /content/drive/My Drive/Classroom/Datasets/num_model.h5
Epoch 3/10
54684/54684 [==============================] - 2111s 39ms/step - loss: 0.0793 - acc: 0.9879 - val_loss: 0.0853 - val_acc: 0.9872

Epoch 00003: val_loss improved from 0.08970 to 0.08527, saving model to /content/drive/My Drive/Classroom/Datasets/num_model.h5
Epoch 4/10
54684/54684 [==============================] - 2107s 39ms/step - loss: 0.0737 - acc: 0.9886 - val_loss: 0.0823 - val_acc: 0.9876

Epoch 

KeyboardInterrupt: ignored

In [0]:
import pickle

# saving spoken token
with open(dr+'number_spoken_tokenizer.pickle', 'wb') as handle:
    pickle.dump(eng_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [0]:
# saving written token
with open(dr+'number_written_tokenizer.pickle', 'wb') as handle:
    pickle.dump(hin_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
